In [1]:
import pandas as pd
from io import StringIO

In [2]:
csv_data = \
'''A,B,C,D
1.0,2.0,3.0,
5.0,6.0,,7.0
10.0,11.0,12.0,
'''

df= pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,NaN
1,5.0,6.0,NaN,7.0
2,10.0,11.0,12.0,NaN


In [3]:
df.isnull()

,A,B,C,D
0,False,False,False,True
1,False,False,True,False
2,False,False,False,True


In [4]:
df.isnull().sum()

A    0
B    0
C    1
D    2
dtype: int64

In [5]:
df.isnull().sum(axis=1)

0    1
1    1
2    1
dtype: int64

In [6]:
df.dropna(axis=0) #행 방향 삭제

,A,B,C,D


In [7]:
df.dropna(axis=1) #열 방향 삭제

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [8]:
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,NaN
1,5.0,6.0,NaN,7.0
2,10.0,11.0,12.0,NaN


In [9]:
df.dropna(thresh=3)

,A,B,C,D
0,1.0,2.0,3.0,NaN
1,5.0,6.0,NaN,7.0
2,10.0,11.0,12.0,NaN


In [10]:
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,NaN
2,10.0,11.0,12.0,NaN


In [11]:
# 평균으로 대체
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr.fit(df.values)
imputed_data = imr.transform(df.values)

In [12]:
imputed_data

array([[ 1. ,  2. ,  3. ,  7. ],
       [ 5. ,  6. ,  7.5,  7. ],
       [10. , 11. , 12. ,  7. ]])

In [13]:
from sklearn.preprocessing import FunctionTransformer
ftr_imr = FunctionTransformer(lambda X:imr.fit_transform(X.T).T, validate=False)
imputed_data = ftr_imr.transform(df.values)
imputed_data

array([[ 1.,  2.,  3.,  2.],
       [ 5.,  6.,  6.,  7.],
       [10., 11., 12., 11.]])

In [14]:
import pandas as pd
df = pd.DataFrame([['green','M',10.1,'class1'],
                 ['red','L',13.5,'class2'],
                  ['blue','XL',15.3,'class1']],
                  columns=['color','size','price','classlabel']
                 )
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [15]:
size_mapping = {
    'XL':3,
    'L':2,
    'M':1
}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [16]:
import numpy as np
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [17]:
df['classlabel'] = df['classlabel'].map(class_mapping)

In [18]:
inv_class_maping = {v:k for k,v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_maping)

In [19]:
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [20]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'])
y

array([0, 1, 0])

In [21]:
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [22]:
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

In [23]:
class_le.classes_

array(['class1', 'class2'], dtype=object)

In [24]:
X = df[['color','size','price']].values

In [25]:
color_le=LabelEncoder()

In [26]:
X[:,0] = color_le.fit_transform(X[:,0])

In [27]:
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [28]:
color_le.classes_

array(['blue', 'green', 'red'], dtype=object)

In [29]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder(dtype=int)
col_trans=ColumnTransformer([('ord_enc',ord_enc,['color'])])
X_trans = col_trans.fit_transform(df)
X_trans

array([[1],
       [2],
       [0]])

In [30]:
col_trans.named_transformers_['ord_enc'].inverse_transform(X_trans)

array([['green'],
       ['red'],
       ['blue']], dtype=object)

In [31]:
ord_enc.categories

'auto'

## 원핫인코딩


In [32]:
type(X[:,0])

numpy.ndarray

In [33]:
from sklearn.preprocessing import OneHotEncoder
X=df[['color','size','price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:,0].reshape(-1,1)).toarray()

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

## ColumnTransformer :  여러 특성을 정수로 변환시키는 변환기
### 매개변수 [인코딩이름, 인코딩객체,변환열]

In [34]:
X

array([['green', 1, 10.1],
       ['red', 2, 13.5],
       ['blue', 3, 15.3]], dtype=object)

In [41]:
from sklearn.compose import ColumnTransformer
X = df[['color','size','price']].values
c_transf = ColumnTransformer([('onehot',OrdinalEncoder(dtype=int),[0]),
                               ('nothing','passthrough',[1,2])
                               ])

array([[0.0, 1.0, 0.0, 1, 10.1],
       [0.0, 0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 0.0, 3, 15.3]], dtype=object)

In [42]:
c_transf.fit_transform(X)

array([[0.0, 1.0, 0.0, 1, 10.1],
       [0.0, 0.0, 1.0, 2, 13.5],
       [1.0, 0.0, 0.0, 3, 15.3]], dtype=object)

In [37]:
X

array([['green', 1, 10.1],
       ['red', 2, 13.5],
       ['blue', 3, 15.3]], dtype=object)

In [38]:
pd.get_dummies(df[['price','color','size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [39]:
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [40]:
pd.get_dummies(df[['price','color','size']],columns=['size'])

,price,color,size_1,size_2,size_3
0,10.1,green,1,0,0
1,13.5,red,0,1,0
2,15.3,blue,0,0,1


In [47]:
color_ohe = OneHotEncoder(categories='auto', drop='first')
#범주형 데이터 자동으로 찾고, 원핫인코딩-> 첫번째 열 삭제
c_transf = ColumnTransformer([
    ('onehot',color_ohe,[0]),
    ('nothing','passthrough',[1,2])
])
c_transf.fit_transform(X)

array([[1.0, 0.0, 1, 10.1],
       [0.0, 1.0, 2, 13.5],
       [0.0, 0.0, 3, 15.3]], dtype=object)